<a href="https://colab.research.google.com/github/nwang2208/Bioinformatics-Class-Projects/blob/mapping-reads/Project2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
k = 21

In [ ]:
!pip install Bio

In [ ]:
from Bio import SeqIO
import math
import numpy as np

Preparing PWM

In [ ]:
pwm=open('/content/drive/MyDrive/122data/PROJECT2A/project2a_PWM.txt','r')
pwm = pwm.readlines()


In [ ]:
pwm_matrix = []

for el in pwm:
    row = []
    num_string= ''
    for i in range(0,len(el)):
        if(el[i] != '\t'):
            if(el[i] == '\n'):
                row.append(float(num_string))
                num_string = ''
            else:
                num_string += el[i]
        elif(el[i] == '\t'):
            row.append(float(num_string))
            num_string = ''
    pwm_matrix.append(row)




Opening seqs

In [ ]:
seq_dict = {}
for seq_record in SeqIO.parse('/content/drive/MyDrive/122data/PROJECT2A/test.fasta', "fasta"):
    seq_dict[seq_record.id] = str(seq_record.seq)


In [ ]:
seq_dict

In [ ]:
r_seq_dict= {}

for key,value in seq_dict.items():
  r_seq = ""
  for x in range(0,len(value)):
    base = ''
    if(value[x] == "A"):
      base = 'T'
    elif(value[x] == "T"):
      base = 'A'
    elif(value[x] == "C"):
      base = 'G'
    else:
      base = 'C'
    r_seq += base
  r_seq_dict[key] = r_seq[::-1]

In [ ]:
r_seq_dict

Go through each seq and find the most probably 21mer

In [ ]:

def score(profile_matrix, motif_list):
    score_list = []
    for el in motif_list:
        score = 1
        for i in range(0, len(el)):

            if(el[i] == 'A'):
                score *= profile_matrix[0][i]
            elif(el[i] == 'C'):
                score *= profile_matrix[1][i]
            elif(el[i] == 'G'):
                score *= profile_matrix[2][i]
            else:
                score *= profile_matrix[3][i]
        score_list.append(score)
    return(score_list)

In [ ]:
motif_list = []
best_list = []
prob_dict = {}
for key, value in seq_dict.items():
    motif_list = []
    for i in range(0, len(value)-21):
        kmer = value[i:i+21]
        motif_list.append(kmer)
    score_list = score(pwm_matrix, motif_list)
    best_score = max(score_list)
    prob_dict[key] = math.log(best_score/(.25**21))
    indexofbest = score_list.index(max(score_list))
    best_motif = motif_list[indexofbest]
    best_list.append(best_motif)


In [ ]:


r_prob_dict = {}
for key, value in r_seq_dict.items():
    motif_list = []
    for i in range(0, len(value)-21):
        kmer = value[i:i+21]
        motif_list.append(kmer)
    score_list = score(pwm_matrix, motif_list)
    best_score = max(score_list)
    r_prob_dict[key] = math.log(best_score/(.25**21))


In [ ]:
final_prob = {}

for key, value in prob_dict.items():
    if(value > r_prob_dict[key]):
        final_prob[key] = value
    else:
        final_prob[key] = r_prob_dict[key]

In [ ]:
final_prob

In [ ]:
sorted_probs = sorted(final_prob.items(), key=lambda x:x[1], reverse=True)
converted_dict = dict(sorted_probs)

converted_dict_keys = list(converted_dict.keys())

predictions = []
for i in range(0, 2000):
    predictions.append(converted_dict_keys[i])

In [ ]:
predictions

In [ ]:
np.savetxt("predictions.csv",
           predictions,
           delimiter =", ",  # Set the delimiter as a comma followed by a space
           fmt ='% s')